# Solvery Mentors — Exploratory Data Analysis

This notebook loads the extracted Solvery mentors dataset, performs EDA with
visualisations, computes summary statistics, and applies TF-IDF + KMeans
clustering on mentor skills.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

# ---------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------

CSV_PATH = Path("solvery_mentors.csv")
CLUSTERED_PATH = Path("solvery_mentors_clustered.csv")

TOP_N_SKILLS = 15
N_CLUSTERS = 5
RANDOM_STATE = 42
KMEANS_N_INIT = 10
TFIDF_MAX_FEATURES = 500
TOP_TERMS_PER_CLUSTER = 8
MIN_EXPERIENCED_SESSIONS = 200
FALLBACK_MIN_SESSIONS = 50

NUMERIC_COLS = ["price", "reviews_count", "sessions_count"]
CORR_COLS = ["price", "sessions_count", "reviews_count"]
DISPLAY_COLS = [
    "name", "title", "company", "price",
    "sessions_count", "reviews_count", "skills",
]


# ---------------------------------------------------------------------------
# Helper functions
# ---------------------------------------------------------------------------


def explode_skills(
    df: pd.DataFrame,
    column: str = "skills",
    top_n: int = TOP_N_SKILLS,
) -> tuple[pd.DataFrame, list[str]]:
    """Explode comma-separated skills into rows; return (top-filtered frame, top skill names)."""
    df_skills = df.dropna(subset=[column]).copy()
    df_skills["skill_list"] = df_skills[column].str.split(r",\s*")
    df_exploded = df_skills.explode("skill_list")
    df_exploded["skill_list"] = df_exploded["skill_list"].str.strip()
    df_exploded = df_exploded[df_exploded["skill_list"].str.len() > 0]
    top = df_exploded["skill_list"].value_counts().head(top_n).index.tolist()
    return df_exploded[df_exploded["skill_list"].isin(top)], top


def plot_median_bar(
    series: pd.Series,
    title: str,
    xlabel: str,
    ylabel: str = "Skill",
    figsize: tuple[int, int] = (12, 6),
) -> None:
    """Plot a horizontal bar chart of a pandas Series."""
    _, ax = plt.subplots(figsize=figsize)
    series.plot(kind="barh", ax=ax)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.tight_layout()
    plt.show()

## 1. Load and prepare data

In [ ]:
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df)} mentors, {len(df.columns)} columns")
df.head()

In [ ]:
for col in NUMERIC_COLS:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.info()
print()
df.describe()

## 2. Distributions

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of price
price_data = df["price"].dropna()
axes[0].hist(price_data, bins=40, edgecolor="black", linewidth=0.5)
axes[0].set_title("Distribution of Price")
axes[0].set_xlabel("Price")
axes[0].set_ylabel("Count")

# Histogram of sessions_count
sessions_data = df["sessions_count"].dropna()
axes[1].hist(sessions_data, bins=40, edgecolor="black", linewidth=0.5)
axes[1].set_title("Distribution of Sessions Count")
axes[1].set_xlabel("Sessions Count")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.show()

## 3. Scatter plots

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(14, 5))

# Price vs Sessions
mask_ps = df["price"].notna() & df["sessions_count"].notna()
axes[0].scatter(df.loc[mask_ps, "price"], df.loc[mask_ps, "sessions_count"], alpha=0.5, s=20)
axes[0].set_title("Price vs Sessions Count")
axes[0].set_xlabel("Price")
axes[0].set_ylabel("Sessions Count")

# Sessions vs Reviews
mask_sr = df["sessions_count"].notna() & df["reviews_count"].notna()
axes[1].scatter(df.loc[mask_sr, "sessions_count"], df.loc[mask_sr, "reviews_count"], alpha=0.5, s=20)
axes[1].set_title("Sessions Count vs Reviews Count")
axes[1].set_xlabel("Sessions Count")
axes[1].set_ylabel("Reviews Count")

plt.tight_layout()
plt.show()

## 4. Median price / sessions by top 15 skills

In [ ]:
df_top, _ = explode_skills(df)

median_price = (
    df_top.groupby("skill_list")["price"]
    .median()
    .sort_values(ascending=False)
)

plot_median_bar(median_price, "Median Price by Top 15 Skills", "Median Price")

print("\nMedian price by top 15 skills:")
print(median_price.to_string())

In [ ]:
median_sessions = (
    df_top.groupby("skill_list")["sessions_count"]
    .median()
    .sort_values(ascending=False)
)

plot_median_bar(median_sessions, "Median Sessions Count by Top 15 Skills", "Median Sessions")

print("\nMedian sessions by top 15 skills:")
print(median_sessions.to_string())

## 5. Correlation matrix

In [ ]:
corr_matrix = df[CORR_COLS].corr()

_, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(corr_matrix.values, aspect="auto")
ax.set_xticks(range(len(CORR_COLS)))
ax.set_yticks(range(len(CORR_COLS)))
ax.set_xticklabels(CORR_COLS, rotation=45, ha="right")
ax.set_yticklabels(CORR_COLS)
ax.set_title("Correlation Matrix")

for i in range(len(CORR_COLS)):
    for j in range(len(CORR_COLS)):
        ax.text(
            j, i, f"{corr_matrix.values[i, j]:.2f}",
            ha="center", va="center", fontsize=12, fontweight="bold",
        )

plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

print("\nCorrelation matrix:")
print(corr_matrix.to_string())

## 6. Top mentors

In [ ]:
print("=" * 80)
print("TOP 20 MENTORS BY SESSIONS")
print("=" * 80)
print(df.nlargest(20, "sessions_count")[DISPLAY_COLS].to_string(index=False))

print("\n" + "=" * 80)
print("TOP 20 MENTORS BY REVIEWS")
print("=" * 80)
print(df.nlargest(20, "reviews_count")[DISPLAY_COLS].to_string(index=False))

print("\n" + "=" * 80)
print("10 CHEAPEST MENTORS WITH > 200 SESSIONS")
print("=" * 80)
experienced = df[df["sessions_count"] > MIN_EXPERIENCED_SESSIONS].nsmallest(10, "price")
if experienced.empty:
    print(
        f"No mentors with > {MIN_EXPERIENCED_SESSIONS} sessions. "
        f"Trying > {FALLBACK_MIN_SESSIONS} sessions \u2026"
    )
    experienced = df[df["sessions_count"] > FALLBACK_MIN_SESSIONS].nsmallest(10, "price")
print(experienced[DISPLAY_COLS].to_string(index=False))

## 7. TF-IDF + KMeans clustering on skills

In [ ]:
df_cluster = df.copy()
df_cluster["skills_text"] = df_cluster["skills"].fillna("").astype(str)

has_skills = df_cluster["skills_text"].str.strip().str.len() > 0
df_with_skills = df_cluster[has_skills].copy()

print(f"Mentors with skills: {len(df_with_skills)} / {len(df_cluster)}")

if len(df_with_skills) >= N_CLUSTERS:
    vectorizer = TfidfVectorizer(
        max_features=TFIDF_MAX_FEATURES,
        token_pattern=r"[^,]+",
        strip_accents="unicode",
        lowercase=True,
    )
    tfidf_matrix = vectorizer.fit_transform(df_with_skills["skills_text"])

    n_clusters = min(N_CLUSTERS, len(df_with_skills))
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=KMEANS_N_INIT)
    clusters = kmeans.fit_predict(tfidf_matrix)

    df_cluster["cluster"] = -1
    df_cluster.loc[has_skills, "cluster"] = clusters

    print("\nCluster distribution:")
    print(df_cluster["cluster"].value_counts().sort_index().to_string())

    feature_names = vectorizer.get_feature_names_out()
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    print("\nTop terms per cluster:")
    for i in range(n_clusters):
        terms = [feature_names[idx].strip() for idx in order_centroids[i, :TOP_TERMS_PER_CLUSTER]]
        print(f"  Cluster {i}: {', '.join(terms)}")
else:
    print("[!] Not enough mentors with skills for clustering.")
    df_cluster["cluster"] = -1

In [ ]:
df_cluster.drop(columns=["skills_text"], errors="ignore").to_csv(CLUSTERED_PATH, index=False)
print(f"Saved clustered data to {CLUSTERED_PATH}  ({len(df_cluster)} records)")

In [ ]:
if df_cluster["cluster"].nunique() > 1:
    _, ax = plt.subplots(figsize=(10, 6))
    for cluster_id in sorted(df_cluster["cluster"].unique()):
        if cluster_id == -1:
            continue
        subset = df_cluster[df_cluster["cluster"] == cluster_id]
        ax.scatter(
            subset["price"],
            subset["sessions_count"],
            label=f"Cluster {cluster_id}",
            alpha=0.6,
            s=25,
        )
    ax.set_title("Mentor Clusters: Price vs Sessions")
    ax.set_xlabel("Price")
    ax.set_ylabel("Sessions Count")
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Only one cluster \u2014 skipping scatter plot.")